In [4]:
## Imports
import tarfile
import urllib
import re

from rich import print, progress

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import spectrum_utils.spectrum as sus
import spectrum_utils.plot as sup

from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor

from hyperopt import fmin, hp, tpe, STATUS_OK

np.random.seed(42)

In [5]:
library_file = "human_hcd_tryp_best.msp"

In [6]:
## Downloading the dataset
url = "https://chemdata.nist.gov/download/peptide_library/libraries/human/HCD/2020_05_19/human_hcd_tryp_best.msp.tar.gz"

# Download file
_ = urllib.request.urlretrieve(url, f"{library_file}.tar.gz")

with tarfile.open(f"{library_file}.tar.gz") as f:
    f.extractall("./datasets/")

In [7]:
dataset_location = "./datasets/" + f"{library_file}"
dataset_location

'./datasets/human_hcd_tryp_best.msp'

In [12]:
## First 10 lines of the dataset
with open(dataset_location, "rt") as f:
    for i, line in enumerate(f):
        print(line.strip())
        if i > 200:
            break

Name: AAAAAAAAAAAAAAAGAGAGAK/2_0

Comment: Consensus Pep=Tryptic Peptype=<Protein><Peptide><Protein> Mods=0 Fullname=R.AAAAAAAAAAAAAAAGAGAGAK.Q 
Charge=2 Parent=798.9263 CE=42.09 NCE=29.43 Q-value=0.0000 Nprot=1 Protein="sp|P55011|S12A2_HUMAN(pre=R,post=Q)" 
Nrep=134/200 Theo_mz_diff=1.2ppm Quality=7/7 MC=0 MCtype=Normal Unassigned_all_20ppm=0.1424 Unassigned_20ppm=0.0416
num_unassigned_peaks_20ppm=44 max_unassigned_ab_20ppm=0.41 top_20_num_unassigned_peaks_20ppm=1/20

Num peaks: 117

110.0712        259243.2        "? 143/200"

115.0864        97764.4 "a2/-1.6ppm 145/200"

...

328.1970        342366.7        "y4-H2O/-2.8ppm,a5/-2.8ppm 198/200"

340.1602        39167.2 "? 103/200"

346.2081        271160.8        "y4/-1.1ppm 198/200"

354.1758        82324.8 "? 143/200"

356.1921        1466974.1       "b5/-2.1ppm,y4-H2O+CO/-2.1ppm 200/200"

In [9]:
def read_msp(filename):
    """Iterate over MSP spectral library file and return spectra as dicts."""
    spectrum = {}
    mz = []
    intensity = []
    annotation = []

    with progress.open(filename, "rt") as f:
        for line in f:
            # `Name: ` is the first line of a new entry in the file
            if line.startswith("Name: "):
                if spectrum:
                    # Finalize and yield previous spectrum
                    spectrum["sequence"] = spectrum["Fullname"].split(".")[1]  # Remove the previous/next amino acids
                    spectrum["mz"] = np.array(mz, dtype="float32")
                    spectrum["intensity"] = np.array(intensity, dtype="float32")
                    spectrum["annotation"] = np.array(annotation, dtype="str")
                    yield spectrum

                    # Define new spectrum
                    spectrum = {}
                    mz = []
                    intensity = []
                    annotation = []

                # Extract everything after `Name: `
                spectrum["Name"] = line.strip()[6:]

            elif line.startswith("Comment: "):
                # Parse all comment items as metadata
                metadata = [i.split("=") for i in line[9:].split(" ")]
                for item in metadata:
                    if len(item) == 2:
                        spectrum[item[0]] = item[1]

            elif line.startswith("Num peaks: "):
                spectrum["Num peaks"] = int(line.strip()[11:])

            elif len(line.split("\t")) == 3:
                # Parse peak list items one-by-one
                line = line.strip().split("\t")
                mz.append(line[0])
                intensity.append(line[1])
                annotation.append(line[2].strip('"'))

    # Final spectrum
    spectrum["sequence"] = spectrum["Fullname"].split(".")[1]  # Remove the previous/next amino acids
    spectrum["mz"] = np.array(mz, dtype="float32")
    spectrum["intensity"] = np.array(intensity, dtype="float32")
    spectrum["annotation"] = np.array(annotation, dtype="str")

In [ ]:
for spectrum in read_msp("human_hcd_tryp_best.msp"):
    print(spectrum["Name"])
    break

In [ ]:
pd.DataFrame({
    "mz": spectrum["mz"],
    "intensity": spectrum["intensity"],
    "annotation": spectrum["annotation"]
})

In [ ]:
plt.figure(figsize=(10,5))

sup.spectrum(
    sus.MsmsSpectrum(
        identifier = spectrum['Name'],
        precursor_mz = float(spectrum['Parent']),
        precursor_charge = int(spectrum['Charge']),
        mz = spectrum['mz'],
        intensity = spectrum['intensity']
    )
)
plt.title(spectrum['Name'])
plt.show()